In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
df_crafts = pd.read_csv('crafts.csv', encoding='utf-8')
display(df_crafts.tail())

,attr,charge,id,mode,name,race,rare,skill1,skill2,skill3
435,NaN,連擊 (Combo) 達 4 或以上,436,1,初心威嚇龍紋,NaN,3,1 回合內，隊伍攻擊力提升 10%,1 回合內，隊伍回復力提升 30%,延遲全體敵人行動 1 回合
436,NaN,連擊 (Combo) 達 4 或以上,437,1,初心圖騰龍紋,NaN,3,1 回合內，隊伍攻擊力提升 10%,1 回合內，隊伍回復力提升 30%,1 回合內，25% 的攻擊傷害無視指定形狀盾
437,NaN,連擊 (Combo) 達 4 或以上,438,1,初心衝鋒龍紋,NaN,3,1 回合內，隊伍攻擊力提升 10%,1 回合內，隊伍回復力提升 30%,1 回合內，25% 的攻擊傷害無視攻前盾
438,NaN,連擊 (Combo) 達 4 或以上,439,1,初心復生龍紋,NaN,3,1 回合內，隊伍攻擊力提升 10%,1 回合內，隊伍回復力提升 30%,1 回合內，完全回復生命力
439,NaN,連擊 (Combo) 達 4 或以上,440,1,初心越時龍紋,NaN,3,1 回合內，隊伍攻擊力提升 10%,1 回合內，隊伍回復力提升 30%,隨機 3 個人類成員的技能 CD 減少 1


## Translate string to value

In [3]:
def Cno_format(no):
    if no < 10:
        str_no = str(no).zfill(2)
    else:
        str_no = str(no)
    return str_no

In [4]:
def series_to_string(craft):
    for i, v in craft.iteritems():
        if isinstance(v, str) is False:
            if math.isnan(v):
                craft[i] = "0"
            else:
                craft[i] = str(craft[i])
    craft["attr"] = attr_to_value(craft["attr"])
    craft["race"] = race_to_value(craft["race"])
    craft["charge"] = charge_to_value(craft["charge"])
    craft["skilltype"] = find_skill_types(craft)
    return craft

In [5]:
def attr_to_value(s):
    attr = ["水", "火", "木", "光", "暗"]
    str2 = 0
    for i in range(5):
        if s == attr[i]:
            str2 = i+1
    str2 = str(str2)
    return str2

In [6]:
def race_to_value(s):
    race = ["人類", "獸類", "神族", "龍類", "妖精類", "魔族", "機械族"]
    str2 = 0
    for i in range(7):
        if s == race[i]:
            str2 = i+1
    str2 = str(str2)
    return str2

In [7]:
def charge_to_value(s):
    charge = ["任何符石", "水符石", "火符石", "木符石", "光符石", "暗符石", "心符石", "發動攻擊", "受到攻擊", "連擊"]
    str2 = 0
    for i in range(10):
        result = re.search(charge[i], s)
        if result is not None:
            str2 = i+1
    str2 = str(str2)
    return str2

## Classifying skill type

In [8]:
def find_skill_type(s):
    skill_set = set()
    skill_type = ['攻擊力(.*)提升', '回復力(.*)提升', '回復(.*)生命力','轉化為我方生命力',
                  '轉化(.*)符石',
                  '屬性相剋', 'CD', 
                  '減少所受傷害', '迴避', '所受傷害不會使你死亡', '延遲',
                  '造成(.*)', '造成(.*)此傷害無視防禦力',
                  '無視(.*)指定形狀盾', '無視(.*)五屬盾', '無視(.*)攻前盾',
                  '防禦力下降', '無視全體敵人(.*)防禦力', '追打', '反擊', '單體攻擊轉化為全體攻擊',
                  '最多(.*)轉化(.*)水符石', '最多(.*)轉化(.*)火符石', '最多(.*)轉化(.*)木符石',
                  '最多(.*)轉化(.*)光符石', '最多(.*)轉化(.*)暗符石', '最多(.*)轉化(.*)心符石', 
                  '直行(.*)水符石', '直行(.*)火符石', '直行(.*)木符石', '直行(.*)光符石', '直行(.*)暗符石', '直行(.*)心符石',
                  '場上的(.*)轉化(.*)水符石', '場上的(.*)轉化(.*)火符石', '場上的(.*)轉化(.*)木符石', 
                  '場上的(.*)轉化(.*)光符石', '場上的(.*)轉化(.*)暗符石', '場上的(.*)轉化(.*)心符石']
    for i in range(len(skill_type)):
        skill = skill_type[i]
        pattern_detail = r'(.*)' + skill + r'(.*)'
        s2 = re.search(pattern_detail, s)
        if s2 is not None:
            skill_set.add(str(i+1))
    return skill_set

In [9]:
def find_skill_types(craft):
    skill_set = set()
    skill_str = craft['skill1']
    skill_set |= find_skill_type(skill_str)
    skill_str = craft['skill2']
    if isinstance(skill_str, str) is True:
        skill_set |= find_skill_type(skill_str)
    skill_str = craft['skill3']
    if isinstance(skill_str, str) is True:
        skill_set |= find_skill_type(skill_str)
    skilltype_str = " ".join(skill_set)
    return skilltype_str

In [10]:
def data_to_wikitext(craft):
    str_Cno = Cno_format(int(craft["id"]))
    str_craft = "<div data-id=\"" + craft["id"] + "\"" + " data-star=\"" + craft["rare"] + "\"" \
    + " data-attribute=\"" + craft["attr"] + "\"" + " data-racialtype=\"" + craft["race"] + "\"" \
    + " data-mode=\"" + craft["mode"] + "\"" + " data-charge=\""  + craft["charge"] + "\"" \
    + " data-skilltype=\"" + craft["skilltype"] + "\"" \
    + " data-skilltype2=\"" + craft["skilltype"] + "\"" \
    + ">{{CraftDesc|C" + str_Cno + "|" + craft["name"] + "}}" + "</div>\n"
    return str_craft

In [11]:
str_all =""
for index, row in df_crafts.iterrows():
    row2 = series_to_string(row)
    str_craft = data_to_wikitext(row2)
#     print(str_craft)
    str_all += str_craft
# print(str_all)


In [12]:
text_file = open("to_wikia_craft.txt", "w")
text_file.write(str_all)
text_file.close()